# Functions for quick bar chart generation

TODO: make charts easier to read. Likely by not using subplots.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.DataFrame(pd.read_csv('../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv'))
df.head()

In [ ]:
# questions = pd.DataFrame(df.T[0].reset_index(drop=False)).rename(columns={'index': 'question_num', 0: 'question'})
# for index, row in questions.iterrows():
#     print(f'-{row.question_num} | {row.question}')

## "Part" questions

In [ ]:
part_questions = []
not_part_questions = []

# [part_questions.append(x.split('_')[0][1:]) for x in df.columns if 'part' in x.lower() and x.split('_')[0][1:] not in part_questions]

for x in df.columns:
    if 'part' in x.lower(): 
        if x.split('_')[0][1:] not in part_questions:
            part_questions.append(x.split('_')[0][1:])
    else:
        if x not in not_part_questions and 'other' not in x.lower():
            not_part_questions.append(x)

print(part_questions)
print(not_part_questions)

In [ ]:
def quick_bar(q_num, ax):
    filter_cols = [col for col in df.columns if col.split('_')[0]==f'Q{q_num}']
    temp_df = df[filter_cols]
    col_names = [x.split('-')[-1] for x in temp_df.iloc[0]]
    title = temp_df.iloc[0][filter_cols[0]].split('-')[0]
    temp_df.columns = col_names
    temp_df = temp_df.notnull().astype('int')
    temp_df = temp_df.iloc[1:].fillna(0)
    temp_df.sum().sort_values(ascending=False).plot.bar(title=title, ax=ax)

In [ ]:
fig, axs = plt.subplots(nrows=len(part_questions), ncols=1, figsize=(15,200))
for i, x in enumerate(part_questions):
    quick_bar(x, axs[i])
plt.subplots_adjust(hspace=1)
plt.show()

## Age

In [ ]:
not_part_questions = []
for x in df.columns:
    if 'part' not in x.lower() and 'other' not in x.lower() and x not in not_part_questions:
        not_part_questions.append(x)
one_part_df = df[not_part_questions]
one_part_df.head(2)

In [ ]:
# not good, redo this hist
age = pd.DataFrame(one_part_df['Q1'][1:])
age['age'] = age['Q1'].str.split('-').str[0].str.replace('+','').astype('int')
age['age'].plot.hist(bins=11, grid=True, title='Bad looking and incorrect age range hist')

## Gender

In [ ]:
one_part_df['Q2'][1:].value_counts().plot.bar(title='Gender')

In [ ]:
one_part_df.iloc[1:].groupby(['Q1','Q2']).size().unstack().plot.bar(stacked=True, 
                                                                    figsize=(15,8),
                                                                    title='Age and gender breakdown')
plt.tight_layout()

## Country data

In [ ]:
one_part_df['Q3'][1:].value_counts().sort_values(ascending=False)[:11].plot.bar(title='Top 10 countries')

In [ ]:
# avg age in each country
top10 = one_part_df.iloc[1:]['Q3'].value_counts()[:11].index.tolist()
one_part_df.loc[one_part_df.Q3.isin(top10)].groupby(['Q3','Q1']).size().unstack().plot.bar(stacked=True, 
                                                                                           figsize=(15,10),
                                                                                           title='Country and age breakdown')
plt.tight_layout()

## Education and career

In [ ]:
one_part_df.iloc[1:].Q4.value_counts().plot.bar(title='Education')

In [ ]:
# avg age in each country
one_part_df.loc[one_part_df.Q3.isin(top10)].groupby(['Q3','Q4']).size().unstack().plot.bar(stacked=True, 
                                                                                           figsize=(15,10),
                                                                                           title='Country and education breakdown')
plt.tight_layout()

In [ ]:
one_part_df.iloc[1:]['Q5'].value_counts().plot.bar(title='Profession')

In [ ]:
index_order = ['I have never written code','< 1 years','1-3 years','3-5 years','5-10 years','10-20 years','20+ years']
one_part_df.Q6.value_counts().reindex(index_order).plot.bar(title='Code exp.')

In [ ]:
def quick_bar2(question):
    q = one_part_df[question]
    t = q.iloc[0]
    q.iloc[1:].value_counts().plot.bar(title=t)
not_part_questions

In [ ]:
quick_bar2('Q8')

In [ ]:
quick_bar2('Q11')

In [ ]:
quick_bar2('Q13')

In [ ]:
quick_bar2('Q15')

In [ ]:
quick_bar2('Q20')

In [ ]:
quick_bar2('Q21')

In [ ]:
quick_bar2('Q22')

In [ ]:
quick_bar2('Q23')

In [ ]:
quick_bar2('Q26')

In [ ]:
quick_bar2('Q28')

In [ ]:
quick_bar2('Q33')

In [ ]:
quick_bar2('Q35')

In [ ]:
quick_bar2('Q41')

# Power BI vs. Tableau

In [ ]:
pbi_df = df.loc[df.Q35=='Microsoft Power BI']
tab_df = df.loc[df.Q35=='Tableau']
print(f'Number of Power BI users: {len(pbi_df)}')
print(f'Number of Tableau users: {len(tab_df)}')

### Businesses

In [ ]:
index_order = ['$0 ($USD)','$1-$99','$100-$999','$1000-$9,999','$10,000-$99,999','$100,000 or more ($USD)']

def compare(question, index_order=[], figsize=(10,5)):
    
    if len(index_order) > 0:
        pbi = pbi_df[question].value_counts().reindex(index_order)
        tab = tab_df[question].value_counts().reindex(index_order)
    else:
        pbi = pbi_df[question].value_counts().sort_values(ascending=False)
        tab = tab_df[question].value_counts().sort_values(ascending=False)

    pd.concat({
        'Power BI': pbi, 
        'Tableau': tab
    }, axis=1).plot.bar(title=df[question][0],figsize=figsize)
    
#     plt.tight_layout()

compare('Q26', index_order)

In [ ]:
index_order = ['0-49 employees','50-249 employees',
               '250-999 employees','1000-9,999 employees',
               '10,000 or more employees']
compare('Q21', index_order)

In [ ]:
compare('Q20')

In [ ]:
index_order = ['0','1-2','3-4','5-9','10-14','15-19','20+']
compare('Q22', index_order)

In [ ]:
compare('Q23')

### Personal exp

In [ ]:
compare('Q28')

In [ ]:
compare('Q33')

In [ ]:
compare('Q41')

In [ ]:
index_order = ['I do not use machine learning methods','Under 1 year',
               '1-2 years','2-3 years','3-4 years','4-5 years',
               '5-10 years','10-20 years','20 or more years']
compare('Q15',index_order)

In [ ]:
index_order = ['Never','Once','2-5 times','6-25 times','More than 25 times']
compare('Q13',index_order)

In [ ]:
compare('Q11')

In [ ]:
compare('Q8')